In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [4]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [7]:
accuracy_STSIMM = []
accuracy_vsp = []
accuracy_mat = []

for i in range(10):
    try:
        X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
        train_test_split(raw_feature, X_STSIMM, label_material, label_visiprog, \
                         test_size=0.5, stratify=label_material)

        res = leave_one_sample_out(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test)
        accuracy_STSIMM.append(res['accuracy'])

        model_mat = LFDA()
        model_mat.fit(X_train, Y_mat_train)
        X_mat_train = model_mat.transform(X_train)
        X_mat_test = model_mat.transform(X_test)       
        res = leave_one_sample_out(X_mat_train, Y_mat_train, X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])

        model_vsp = LFDA_VISIPROG()
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_train = model_vsp.transform(X_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_train, Y_mat_train, X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except:
        print("Skipping trial {}".format(i))
        pass


INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:vis

In [8]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSTIMM':accuracy_STSIMM})
display(df)
display(df.mean())

,STSTIMM,material,visiprog
0,0.818147,0.982844,0.980938
1,0.818528,0.981700,0.977507
2,0.822341,0.978650,0.971407
3,0.811285,0.981700,0.974075
4,0.812047,0.980938,0.974075
5,0.811285,0.975982,0.974457
6,0.813191,0.973694,0.972551
7,0.806710,0.975600,0.972932
8,0.827297,0.984750,0.975600
9,0.821960,0.987038,0.979794


STSTIMM     0.816279
material    0.980290
visiprog    0.975334
dtype: float64

In [ ]:
print(Y_mat_test)